# 프롬프트 (Prompt)
`02_prompt.ipynb`
- LLM한테 주는 입력(지시, 맥락, 기록)
    - 지시: '~~해줘'
    - 맥락: '내가 어제 피자를 한 판 먹었어. 근데 체헸어. 어떻게 해야함?'
    - 기억: Memory - 지금까지 했던 대화 내용
    

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model = 'gpt-4.1-nano')

In [ ]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 매서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke로 쓰게 됨)
prompt.format(country = '대한민국')

'대한민국의 수도가 어디인가요?'

In [5]:
chain = prompt | llm
chain.invoke({'country':'대한민국'}).content

'대한민국의 수도는 서울입니다.'

# Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [8]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template = template,
    input_variables = ['c1'],
    partial_variables={
        'c2':'미국'
    }
)

prompt.format(c1 = '한국')

'한국과 미국의 수도는 각각 어디인가요?'

In [ ]:
chain = prompt | llm
chain.invoke({'c1':'kor','c2':'us'}).content

'네, 한국과 미국의 수도는 다음과 같습니다:\n\n- 대한민국(한국)의 수도는 서울입니다.\n- 미국의 수도는 워싱턴 D.C.입니다.'

In [18]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    templete = '오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

ValidationError: 1 validation error for PromptTemplate
template
  Field required [type=missing, input_value={'templete': '오늘 날...at 0x0000013ED878BBA0>}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

## `PromptTemplate`

- 단순 1회성 명령을 내린다.

In [ ]:
chain = prompt | llm
print(chain.invoke({'n':3}).content)


In [15]:
prompt.format(n = 3)

KeyError: 'c1'

# `chatPromptTemplate`
- 채팅을 주고받는 템플릿 생성용
- 대화목록을 LLM에게 주입
- 하나의 chat은 `role` 과 `message`로 구성됨

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country = '한국')

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디?'), additional_kwargs={})])

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        #role - message
        ('system','당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human','반가워!'),
        ('ai','무엇을 도와드릴까요?'),
        ('human','{user_input}')
    ]
)

messages_str = chat_template.format(name = 'gaida', user_input = '이름이 뭐니?')
messages_cls = chat_template.format_messages(name = 'gaida', user_input = '이름이 뭐니?')  
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida이야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida이야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [25]:
llm.invoke(messages_cls).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'안녕하세요! 저는 gaida라고 해요. 만나서 반가워요!'

In [26]:
chain = chat_template | llm | StrOutputParser()

res = chain.invoke({'name':'gaida', 'user_input' :'내 이름은 유태영. 너와 나의 이름점을 봐줘'})
print(res)

반갑습니다, 유태영님! 저는 이름점 분석을 통해 이름이 가진 의미나 성격적 특성을 함께 살펴볼 수 있어요. 이름에 담긴 의미, 발음, 또는 이름의 구성에 따라 다양한 해석이 가능하답니다. 혹시 유태영이라는 이름에 특별히 궁금한 점이나, 이름에 담긴 의미를 알고 싶으신 부분이 있으신가요?


## 프롬프팅
내가 원하는 답변을 구구절절 설명 x-> 예시를 주고 답변의 형태를 유도

- Zero-Shot prompting: 예시 없이 질문만 -> 바로 답변
- One-shot prompting: 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting** : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

### `다른 좋은 프롬프트 활용하기 (langchain-hub)`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [ ]:
from langchain import hub
prompt = hub.pull('hwchase17/react')
print(prompt)

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [28]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'
)
hub.push('gaida/prompt-hub-test',prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: gaida